In [1]:
import numpy as np
from tqdm.notebook import tqdm

from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.apis import init_detector, inference_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

from xml_to_np import xml_to_np, all_xml_to_np
from AP import get_single_image_results
from soft_nms import py_cpu_softnms
from dbscan_result_to_aligned_result import dbscan_result_to_aligned_result
from utils import detect_quads, get_free_gpu

In [2]:
checkpoint_file = '/home/aiarhipov/centernet/exps/16_paper_params_dla34_batch8/epoch_150.pth'

config_file = "/home/aiarhipov/centernet/exps/16_paper_params_dla34_batch8/config.py"
cfg = Config.fromfile(config_file)

set_random_seed(0, deterministic=False)

# Build dataset
dataset = build_dataset(cfg.data.test)

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = dataset.CLASSES

model = init_detector(config_file, checkpoint_file, device='cuda:1')
# img_path = '/home/aiarhipov/centernet/imgs/0d995a743dccbda88af4955cd07ab5bf3b8b663e.jpg'  # or img = mmcv.imread(img), which will only load it once
# res = inference_detector(model, img_path)
# res1 = res
# threshold = 0.5
# show_result_pyplot(model, img_path, res, score_thr = threshold, out_file = "/home/aiarhipov/centernet/imgs/output.jpg")
# res_algo2 = algo2_result_to_aligned_result(res1, threshold=threshold, r=0.2)
# res_dbscan = dbscan_result_to_aligned_result(res1, threshold=threshold)
# _res_algo2 = res_algo2
# _res_dbscan = res_dbscan
# show_result_pyplot(model, img_path, _res_algo2, score_thr = threshold, out_file = "/home/aiarhipov/centernet/imgs/output_algo2.jpg")
# show_result_pyplot(model, img_path, _res_dbscan, score_thr = threshold, out_file = "/home/aiarhipov/centernet/imgs/output_dbscan.jpg")

loading annotations into memory...
Done (t=5.72s)
creating index...
index created!
load checkpoint from local path: /home/aiarhipov/centernet/exps/16_paper_params_dla34_batch8/epoch_150.pth


2023-06-19 17:19:51,795 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.0.conv is upgraded to version 2.
2023-06-19 17:19:51,798 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.2.conv is upgraded to version 2.
2023-06-19 17:19:51,800 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.4.conv is upgraded to version 2.


In [3]:
# conf_threshold = 0.5
# iou_threshold = 0.9
# for idx in range(int(len(dataset) / 100)):
#     anno = dataset.get_ann_info(idx=idx)
#     boxes = anno["bboxes"]
#     segm_path = anno["seg_map"]
#     xml_path = f"/home/aiarhipov/datasets/WTW-dataset/test/xml/{segm_path[:-4]}.xml"
#     img_path = f"/home/aiarhipov/datasets/WTW-dataset/test/images/{segm_path[:-4]}.jpg"
    
#     gt_boxes = xml_to_np(xml_path)

#     pred = inference_detector(model, img_path)[0]
    
#     conf_pred = pred[pred[:, 4] > conf_threshold]
#     print("conf = ", conf_pred.shape[0])
#     print("gt = ", gt_boxes.shape[0])
    
#     map_pred_to_gt_idx = []
#     map_pred_to_gt_iou = []
#     for idx_pred, pred_box in enumerate(conf_pred):
#         best_match_idx = None
#         best_match_iou = 0
#         for idx_gt, gt_box in enumerate(gt_boxes):
#             iou = calc_iou_individual(pred_box[:4].tolist(), gt_box[:4].tolist())
#             if iou > best_match_iou and iou > iou_threshold:
#                 best_match_iou = iou
#                 best_match_idx = idx_gt
#         map_pred_to_gt_idx.append(best_match_idx)
#         map_pred_to_gt_iou.append(best_match_iou)
    
#     tp = len([x for x in map_pred_to_gt_idx if str(x) != "nan"])
#     print("len map_pred_to_gt_idx or tp ", tp)
#     print("map_pred_to_gt_iou ", map_pred_to_gt_iou)
#     print(f"precision = {tp / conf_pred.shape[0]}")
#     print(f"recall = {tp / gt_boxes.shape[0]}")

# Physical structure

In [4]:
# conf_threshold = 0.5
# iou_threshold = 0.9
# res = []
# for idx in range(int(len(dataset))):
#     anno = dataset.get_ann_info(idx=idx)
#     boxes = anno["bboxes"]
#     segm_path = anno["seg_map"]
#     xml_path = f"/home/aiarhipov/datasets/WTW-dataset/test/xml/{segm_path[:-4]}.xml"
#     img_path = f"/home/aiarhipov/datasets/WTW-dataset/test/images/{segm_path[:-4]}.jpg"
    
#     gt_boxes = xml_to_np(xml_path)
    
#     pred = inference_detector(model, img_path)[0]
    
#     conf_indexes = py_cpu_softnms(pred[:, :4], pred[:, 4], thresh=0.475, method=2)
#     # print(pred[conf_indexes])
#     conf_pred = pred[conf_indexes]
#     # conf_pred = pred[pred[:, 4] > conf_threshold]
#     # conf_pred = pred[:int(gt_boxes.shape[0] * 1.5)]
#     metrics = get_single_image_results(gt_boxes[:, :4].tolist(), conf_pred[:, :4].tolist(), iou_thr=iou_threshold)
    
#     if (metrics["true_pos"] + metrics["false_pos"]) != 0:
#         metrics["Precision"] = metrics["true_pos"] / (metrics["true_pos"] + metrics["false_pos"])
#     else:
#         metrics["Precision"] = 0
        
#     if (metrics["true_pos"] + metrics["false_neg"]) != 0:
#         metrics["Recall"] = metrics["true_pos"] / (metrics["true_pos"] + metrics["false_neg"])
#     else:
#         metrics["Recall"]
        
#     if (metrics["Precision"] + metrics["Recall"]) != 0:
#         metrics["F1"] = 2 * metrics["Precision"] * metrics["Recall"] / (metrics["Precision"] + metrics["Recall"])
#     else:
#         metrics["F1"] = 0
        
#     res.append(metrics)
    
# Precision = np.mean([d["Precision"] for d in res])
# Recall = np.mean([d["Recall"] for d in res])
# F1 = np.mean([d["F1"] for d in res])

# print(f"Precision = {Precision}")
# print(f"Recall = {Recall}")
# print(f"F1 = {F1}")

# dbscan before Soft-nms

In [5]:
conf_threshold = 0.5
iou_threshold = 0.9

res = []
for idx in tqdm(range(int(len(dataset)))):
    anno = dataset.get_ann_info(idx=idx)
    boxes = anno["bboxes"]
    segm_path = anno["seg_map"]
    xml_path = f"/home/aiarhipov/datasets/WTW-dataset/test/xml/{segm_path[:-4]}.xml"
    img_path = f"/home/aiarhipov/datasets/WTW-dataset/test/images/{segm_path[:-4]}.jpg"
    
    gt_boxes = xml_to_np(xml_path)
    pred = inference_detector(model, img_path)[0]
    conf_indexes = py_cpu_softnms(pred[:, :4], pred[:, 4], thresh=0.475, method=2)
    # print(f"conf_indexes = {conf_indexes}")
    conf_pred = pred[conf_indexes]
    # print(f"conf_pred[0] = {conf_pred[:5]}")
    if conf_pred.shape[0] > 1:
        conf_pred = dbscan_result_to_aligned_result([conf_pred])
        if conf_pred:
            conf_pred = conf_pred[0]
    # print(f"conf_pred[0] = {conf_pred[:5]}")
    # conf_pred = pred[pred[:, 4] > conf_threshold]
    # conf_pred = pred[:int(gt_boxes.shape[0] * 1.5)]
    # print("#"*100)
            metrics = get_single_image_results(gt_boxes[:, :4].tolist(), conf_pred[:, :4].tolist(), iou_thr=iou_threshold)
            
            if (metrics["true_pos"] + metrics["false_pos"]) != 0:
                metrics["Precision"] = metrics["true_pos"] / (metrics["true_pos"] + metrics["false_pos"])
            else:
                metrics["Precision"] = 0
                
            if (metrics["true_pos"] + metrics["false_neg"]) != 0:
                metrics["Recall"] = metrics["true_pos"] / (metrics["true_pos"] + metrics["false_neg"])
            else:
                metrics["Recall"] = 0
                
            if (metrics["Precision"] + metrics["Recall"]) != 0:
                metrics["F1"] = 2 * metrics["Precision"] * metrics["Recall"] / (metrics["Precision"] + metrics["Recall"])
            else:
                metrics["F1"] = 0
                
            res.append(metrics)

Precision = np.mean([d["Precision"] for d in res])
Recall = np.mean([d["Recall"] for d in res])
F1 = np.mean([d["F1"] for d in res])

print(f"Precision = {Precision}")
print(f"Recall = {Recall}")
print(f"F1 = {F1}")

  0%|          | 0/3611 [00:00<?, ?it/s]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

### QUADS IOU

In [ ]:
from shapely.geometry import box, Polygon
import numpy as np

# Define Each polygon 
pol1_xy = np.array([[130, 27], [129.52, 27], [129.45, 27.1], [130.13, 26]])
pol2_xy = np.array([[30, 27.200001], [129.52, 27.34], [129.45, 27.1], [130.13, 26.950001]])
polygon1_shape = Polygon(pol1_xy)
polygon2_shape = Polygon(pol2_xy)

# Calculate Intersection and union, and tne IOU
polygon_intersection = polygon1_shape.intersection(polygon2_shape).area
polygon_union = polygon1_shape.union(polygon2_shape).area
IOU = polygon_intersection / polygon_union 

quad = np.array(range(9))
quad = quad[:-1]
quad.reshape((-1, 2))

### QUADS PHYSICAL STRUCTURE

In [1]:
import numpy as np
from tqdm.notebook import tqdm

from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.apis import init_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

from xml_to_np import xml_to_np, all_xml_to_np
from AP import get_single_image_results
from soft_nms import py_cpu_softnms
from utils import detect_quads, get_free_gpu
conf_threshold = 0.1
iou_threshold = 0.9

checkpoint_file="/home/aiarhipov/centernet/exps/32_quad_long/latest.pth"
config_file = "/home/aiarhipov/centernet/exps/32_quad_long/config.py"
cfg = Config.fromfile(config_file)
set_random_seed(0, deterministic=False)
dataset = build_dataset(cfg.data.test)


# model = init_detector(config_file, checkpoint_file, device=f"cpu")
model = init_detector(config_file, checkpoint_file, device=f"cuda:{get_free_gpu()}")
model.CLASSES = ("box",)

res = []
for idx in tqdm(range(int(len(dataset)))):
    anno = dataset.get_ann_info(idx=idx)
    boxes = anno["bboxes"]
    segm_path = anno["seg_map"]
    xml_path = f"/home/aiarhipov/datasets/WTW-dataset/test/xml/{segm_path[:-4]}.xml"
    img_path = f"/home/aiarhipov/datasets/WTW-dataset/test/images/{segm_path[:-4]}.jpg"
    
    gt_boxes = all_xml_to_np(xml_path)

    quads = detect_quads(img_path, model, integer=False)
    quads = quads[quads[:, 8] > conf_threshold]
    
    boxes = np.vstack([(quads[:, 0] + quads[:, 6]) / 2, 
                       (quads[:, 1] + quads[:, 3]) / 2, 
                       (quads[:, 2] + quads[:, 4]) / 2, 
                       (quads[:, 5] + quads[:, 7]) / 2]).T
    
    nms_indexes = py_cpu_softnms(boxes, quads[:, 8], thresh=0.475, method=2)
    nms_quads = quads[nms_indexes]

    metrics = get_single_image_results(gt_boxes[:, :8], nms_quads[:, :8], iou_thr=iou_threshold, quad=True)
    
    if (metrics["true_pos"] + metrics["false_pos"]) != 0:
        metrics["Precision"] = metrics["true_pos"] / (metrics["true_pos"] + metrics["false_pos"])
    else:
        metrics["Precision"] = 0
        
    if (metrics["true_pos"] + metrics["false_neg"]) != 0:
        metrics["Recall"] = metrics["true_pos"] / (metrics["true_pos"] + metrics["false_neg"])
    else:
        metrics["Recall"] = 0
        
    if (metrics["Precision"] + metrics["Recall"]) != 0:
        metrics["F1"] = 2 * metrics["Precision"] * metrics["Recall"] / (metrics["Precision"] + metrics["Recall"])
    else:
        metrics["F1"] = 0
        
    res.append(metrics)

    Precision = np.mean([d["Precision"] for d in res])
    Recall = np.mean([d["Recall"] for d in res])
    F1 = np.mean([d["F1"] for d in res])

    print(f"Precision = {Precision}, Recall = {Recall}, F1 = {F1}")
Precision = np.mean([d["Precision"] for d in res])
Recall = np.mean([d["Recall"] for d in res])
F1 = np.mean([d["F1"] for d in res])

print(f"Precision = {Precision}")
print(f"Recall = {Recall}")
print(f"F1 = {F1}")

loading annotations into memory...
Done (t=11.31s)
creating index...
index created!
Using GPU=6
load checkpoint from local path: /home/aiarhipov/centernet/exps/32_quad_long/latest.pth


2023-06-19 17:20:24,845 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.0.conv is upgraded to version 2.
2023-06-19 17:20:24,848 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.2.conv is upgraded to version 2.
2023-06-19 17:20:24,849 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.4.conv is upgraded to version 2.


  0%|          | 0/3611 [00:00<?, ?it/s]

KeyError: 'batch_input_shape'